In [1]:
import numpy as np
import pandas as pd
import os
from pandas import DataFrame,Series
import matplotlib.pyplot as plt
%matplotlib inline

import sys
argvs = sys.argv

def dist(d):
    if d > 300:
        d -= 600;
    elif d < -300:
        d += 600
    return d

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)  



# Move Data Preprocessing

In [ ]:
#why 18, there were only 34 = 2*17 participants???
for exp_num in range(1,19):
    
    print("analysing Exp"+str(exp_num))
    
    os.mkdir("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift/Exp"+str(exp_num))
    
# pos might be the 15 trials each team was doing
    for pos_num in range(0,15):

        vel1 = [0]*6000
        vel2 = [0]*6000

        acc1 = [0]*6000
        acc2 = [0]*6000

        turn1 = [0]*6000
        turn2 = [0]*6000

        sign1 = [0]*6000
        sign2 = [0]*6000

        filename = "C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/rawdata/Exp"+str(exp_num)+"/pos"+str(pos_num)+".dat" 
        f = open(filename, 'r')
        outfile = "C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift/Exp"+str(exp_num)+"/pos"+str(pos_num)+".m_s_c"
        fo = open(outfile, 'w')


        prev_p1pos = prev_p2pos = 0
        prev_p1vel = prev_p2vel = 0
        p1_move = p2_move = 0
        p1acc = p2acc =0
        p1_count = 0
        p2_count = 0
        step = 0
        for line in f:
            itemList = line[:-1].split('\t')
            time = itemList[0]
            p1_click = int(itemList[5])
            p2_click = int(itemList[6])
            p1_sti = int(itemList[9])
            p2_sti = int(itemList[10])
            p1pos = float(itemList[1])
            p2pos = float(itemList[2])   

            p1vel = dist(p1pos - prev_p1pos)
            p2vel = dist(p2pos - prev_p2pos)     
            p1acc = p1vel - prev_p1vel
            p2acc = p2vel - prev_p2vel

            vel1[step]= p1vel
            vel2[step]= p2vel


            acc1[step]= p1acc
            acc2[step]= p2acc

            turn1[step] = p1vel - prev_p1vel
            turn2[step] = p2vel - prev_p2vel

            if(vel1[step] == 0):
                if(step == 0):
                    sign1[step] = 0
                else:
                    sign1[step] = sign1[step-1]
            elif(vel1[step] > 0):
                sign1[step] = 1
            else:
                sign1[step] = -1

            if(vel2[step] == 0):
                if(step == 0):
                    sign2[step] = 0
                else:
                    sign2[step] = sign2[step-1]
            elif(vel2[step] > 0):
                sign2[step] = 1
            else:
                sign2[step] = -1



            if(step==0):
                turn1[step]=0
                turn2[step]=0
            else:        
                if (sign1[step]*sign1[step-1] < 0):
                    turn1[step] = 1
                else:
                    turn1[step] = 0

                if (sign2[step]*sign2[step-1] < 0):
                    turn2[step] = 1
                else:
                    turn2[step] = 0


            prev_p1pos = p1pos
            prev_p2pos = p2pos
            prev_p1vel = p1vel
            prev_p2vel = p2vel
            fo.write("%d %d %d %d %d %d %d" % (step, turn1[step], p1_sti, p2_sti, turn2[step], p1_click, p2_click))
            fo.write("\n")
            step = step + 1
        f.close()
        fo.close()

In [ ]:
try:
    os.mkdir("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift_diff")
except:
    print("dir already there")

for exp_num in range(1,19):
    
    print("analysing Exp"+str(exp_num))
    
    try:
        os.mkdir("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift_diff\Exp"+str(exp_num))
    except:
        print("file already exists")
        
    for pos_num in range(0,15):
        d_turn = pd.read_csv("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift/Exp"+str(exp_num)+"/pos"+str(pos_num)+".m_s_c",sep=' ',header=None,index_col=0)
        d = pd.read_csv("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/situation/Exp"+str(exp_num)+"/pos"+str(pos_num)+"_sep.csv",index_col=0)

        stim_dif1 = abs(d['9'].diff()).copy()
        stim_dif2 = abs(d['10'].diff()).copy()

        stim_dif1.loc[0] = 0
        stim_dif2.loc[0] = 0

        output = DataFrame([[0]*len(d)])
        output.loc[0] = np.array(d_turn[1])
        output.loc[1] = np.array(stim_dif1)
        output.loc[2] = np.array(stim_dif2)
        output.loc[3] = np.array(d_turn[4])
        output.loc[4] = np.array(d['5'])
        output.loc[5] = np.array(d['6'])

        output = output.T
        output.loc[0] = np.array([0]*6)
        output = output.astype(int)
        output.to_csv("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift_diff/Exp"+str(exp_num)+"/pos"+str(pos_num)+".m_s_c",sep =' ',header = None)


# Calculation of local TE

In [2]:
#from jpype import *
import jpype
import random
jarLocation = "C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/infodynamics/infodynamics.jar"
# Change location of jar to match yours:
#jarLocation = "/Users/Hiroki/Desktop/infodynamics/infodynamics.jar"
# Start the JVM (add the "-Xmx" option with say 1024M if you get crashes due to not enough memory space)
jpype.startJVM(jpype.getDefaultJVMPath(), "-ea", "-Djava.class.path=" + jarLocation)
#startJVM(getDefaultJVMPath(), "-ea", "-Djava.class.path=" + jarLocation)

c:\users\david-bierbrauer\appdata\local\programs\python\python38\lib\site-packages\jpype\_core.py:201: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  warnings.warn("""


In [4]:
sampling_rate = 5
try:
    os.mkdir('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy')
    os.mkdir('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds'+str(sampling_rate)+'_diff')
except:
    print("folders exist already")
for exp_num in range(1,19):
    try:
        os.mkdir('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds'+str(sampling_rate)+'_diff'+'/Exp'+str(exp_num))
    except:
        print("folders exist already")
    print(exp_num)

    arr = [[0]*7]
    past_len = 1
    for pos_num in range(0,15):
        try:
            os.mkdir('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds'+str(sampling_rate)+'_diff'+'/Exp'+str(exp_num)+'/pos'+str(pos_num))
        except:
            print("folder exists")
            
        datafile = 'C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift_diff/Exp'+str(exp_num)+'/pos'+str(pos_num)+'.m_s_c'
        _arr = np.loadtxt(datafile,dtype = int)        
                
        arr_2_len = int(len(_arr)/sampling_rate)
        arr_2 = np.array([([0]*7)]*arr_2_len)
        for i in range(0,arr_2_len):
            arr_2[i] = _arr[sampling_rate*i:sampling_rate*(i+1)].max(axis=0)
        arr = arr_2
        


        k = 1
        l = [1,2,3,4]

        ave = np.array([[0.0]*16]*4)

        for l_num in range(3,4):

            k = l[l_num]
            b = np.array([[0.0]*16]*len(arr))

            for i in range(0,16):
                source_num = int(i/4) + 1
                dest_num = i%4 + 1
                
                teCalcClass = jpype.JPackage("infodynamics.measures.discrete").TransferEntropyCalculatorDiscrete
                teCalc = teCalcClass(2,k,l[l_num])
                #teCalc = teCalcClass(2,k,l[l_num])
                teCalc.initialise()
                teCalc.addObservations(arr[:,source_num], arr[:,dest_num])
                ave[l_num,i] = teCalc.computeAverageLocalOfObservations()
                a = teCalc.computeLocalFromPreviousObservations(arr[:,source_num], arr[:,dest_num])

                for j in range(0,len(arr)):
                    b[j,i] = a[j]

            
            np.savetxt('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds'+str(sampling_rate)+'_diff'+'/Exp'+str(exp_num)+'/pos'+str(pos_num)+'/TE_'+str(k)+'_'+str(l[l_num])+'.txt',b)
        
        np.savetxt('C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds'+str(sampling_rate)+'_diff'+'/Exp'+str(exp_num)+'/pos'+str(pos_num)+'/average_k_l.txt',ave)



folders exist already
folders exist already
1
folder exists
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpyp

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

...

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class 'jpype._jclass._JClass'>
<class '

# Plot

In [7]:
# まとめ1004
numSize=14
fig = plt.figure(figsize=(12,8))
subCount = 1


for windowSize in [1,5,10]:

    TE_PAS_1 = DataFrame([[0.0]*4])
    TE_PAS_2 = DataFrame([[0.0]*4])

    for PAS in range(1,5):
        TEbefore = DataFrame([[0]*16])
        TEafter = DataFrame([[0]*16])

        window_s = windowSize*100
        count = 0

        for exp_num in range (1,19):

            for pos_num in range(0,15):

                TE = pd.read_csv("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/TransferEntropy/ds5_diff/Exp"+str(exp_num)+"/pos"+str(pos_num)+"/TE_4_4.txt",sep=' ',header=None)
                d_diff = pd.read_csv("C:/Users/david-bierbrauer/Documents/LabRotation/FroeseUnit/LRshift_diff/Exp%d/pos%d.m_s_c" % (exp_num,pos_num),sep = ' ',header = None,index_col = 0)
                print(TE)
                df = df2[df2['Exp']==exp_num][df2['pos']==pos_num]

                clickTime1 = 0
                clickTime2 = 0

                if d_diff[5].sum()>0:
                    clickTime1 = d_diff[d_diff[5]==1].index[0]
                    if clickTime1>window_s and clickTime1+window_s<6000 and int(df[df['player']==1]['PAS'])==PAS:
                        TEbefore.loc[count] = np.array(TE.loc[(clickTime1-window_s)//5:clickTime1//5].mean())
                        TEafter.loc[count] = np.array(TE.loc[clickTime1//5:(clickTime1+window_s)//5].mean())

                        count = count+1

                if d_diff[6].sum()>0:
                    clickTime2 = d_diff[d_diff[6]==1].index[0]
                    if clickTime2>window_s and clickTime2+window_s<6000 and int(df[df['player']==2]['PAS'])==PAS:
                        TEbefore.loc[count] = np.array(TE.loc[(clickTime2-window_s)//5:clickTime2//5].mean())[::-1]
                        TEafter.loc[count] = np.array(TE.loc[clickTime2//5:(clickTime2+window_s)//5].mean())[::-1]

                        count = count+1
        _TE_PAS_1 = np.array([(TEbefore[1]).mean(),(TEbefore[1]).std()/np.sqrt(len(TEbefore[1])),(TEafter[1]).mean(),(TEafter[1]).std()/np.sqrt(len(TEbefore[1]))])
        _TE_PAS_2 = np.array([(TEbefore[13]).mean(),(TEbefore[13]).std()/np.sqrt(len(TEbefore[13])),(TEafter[13]).mean(),(TEafter[13]).std()/np.sqrt(len(TEbefore[13]))])


        TE_PAS_1.loc[PAS-1] = _TE_PAS_1
        TE_PAS_2.loc[PAS-1] = _TE_PAS_2
        
    fig.add_subplot(2,3,4-subCount)
    width = 0.2
    ax=fig.gca()
    yellow = ax.bar(2+np.arange(3)-width*0.5-0.2,TE_PAS_1[0][1:4],width,yerr = np.array(TE_PAS_1[1][1:4]),color = tableau20[16],ecolor = tableau20[16])
    green = ax.bar(2+np.arange(3)-width*0.5+0.2,TE_PAS_2[0][1:4],width,yerr = np.array(TE_PAS_2[1][1:4]),color = tableau20[4],ecolor = tableau20[4])


    plt.axhline(0,color= 'k')
    plt.title('before click ('+str(windowSize)+'s)',fontsize=15)
    plt.xticks(np.linspace(1, 5, 5, endpoint= False,dtype = int))
    plt.xlim(1.5,4.5)
    plt.xticks(fontsize=numSize)
    plt.yticks(fontsize=numSize)
    plt.ylim(0,0.18)
    if(subCount == 1):
        ax.legend((yellow[0], green[0]), ('M1'+r'$\to$'+'S1', 'M2'+r'$\to$'+'S1'),bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)




    fig.add_subplot(2,3,subCount+3)
    ax=fig.gca()
    yellow = ax.bar(2+np.arange(3)-width*0.5-0.2,TE_PAS_1[2][1:4],width,yerr=np.array(TE_PAS_1[3][1:4]),color = tableau20[16],ecolor = tableau20[16])
    green = ax.bar(2+np.arange(3)-width*0.5+0.2,TE_PAS_2[2][1:4],width,yerr=np.array(TE_PAS_2[3][1:4]),color = tableau20[4],ecolor = tableau20[4])

    plt.axhline(0,color= 'k')
    plt.title('after click ('+str(windowSize)+'s)',fontsize=15)
    plt.xlabel('PAS',fontsize=15)
    plt.xticks(np.linspace(1, 5, 5, endpoint= False,dtype = int))
    plt.xlim(1.5,4.5)
    plt.xticks(fontsize=numSize)
    plt.yticks(fontsize=numSize)
    plt.ylim(0,0.18)
    subCount = subCount + 1


       0         1         2         3         4    5         6         7   \
0     0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
1     0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
2     0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
3     0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.000000   
4     0.0  0.025974  0.008412  0.001980  0.010807  0.0 -0.007475  0.014796   
...   ...       ...       ...       ...       ...  ...       ...       ...   
1192  0.0 -0.053286 -0.043297  0.011713 -0.024200  0.0 -0.011588 -0.387023   
1193  0.0  0.052467  0.028599  0.037292  0.000000  0.0  0.269607  0.161281   
1194  0.0 -0.020759  0.008412  0.001980 -0.029747  0.0 -0.005538  0.016852   
1195  0.0  0.011809  0.008412  0.001980  0.302029  0.0 -0.062122 -0.036092   
1196  0.0  0.060474 -0.089005  0.064653  0.111893  0.0  0.152003 -0.072851   

            8         9    10        11        12        13    

NameError: name 'df2' is not defined

<Figure size 864x576 with 0 Axes>